<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-the-stocks" data-toc-modified-id="Loading-the-stocks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading the stocks</a></span></li><li><span><a href="#Getting-the-AI-started" data-toc-modified-id="Getting-the-AI-started-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Getting the AI started</a></span><ul class="toc-item"><li><span><a href="#Regression" data-toc-modified-id="Regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Regression</a></span></li></ul></li></ul></div>

# AI Part

In [1]:
# cell for autoreload includes
%load_ext autoreload
%autoreload 2

## Loading the stocks

In [1]:
from src.stocks import Stocks
# The warnings are due to the includes in the file

22/06/14 12:18:17 WARN Utils: Your hostname, Mathieus-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.27.99.57 instead (on interface feth4588)
22/06/14 12:18:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/14 12:18:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/14 12:18:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql.types import DoubleType, StringType, DateType, StructType, StructField

In [3]:
data_schema = StructType([
    StructField('Date', DateType(), True),
    StructField('High', DoubleType(), True),
    StructField('Low', DoubleType(), True),
    StructField('Open', DoubleType(), True),
    StructField('Close', DoubleType(), True),
    StructField('Volume', DoubleType(), True),
    StructField('Adj Close', DoubleType(), True),
    StructField('company_name', StringType(), True)
])

In [4]:
stocks = Stocks(header=True, delimiter=',', schema=data_schema)

In [10]:
df = stocks.stocks[0].df

## Getting the AI started

In [15]:
type(df)

pyspark.sql.dataframe.DataFrame

In [19]:
display(df)

DataFrame[Date: date, High: double, Low: double, Open: double, Close: double, Volume: double, Adj Close: double, company_name: string]

In [18]:
df.show()

+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+
|      Date|             High|              Low|             Open|            Close|   Volume|        Adj Close|company_name|
+----------+-----------------+-----------------+-----------------+-----------------+---------+-----------------+------------+
|2017-01-03|789.6300048828125|775.7999877929688|778.8099975585938|786.1400146484375|1657300.0|786.1400146484375|      GOOGLE|
|2017-01-04|791.3400268554688|783.1599731445312|788.3599853515625|786.9000244140625|1073000.0|786.9000244140625|      GOOGLE|
|2017-01-05|  794.47998046875|  785.02001953125|786.0800170898438|  794.02001953125|1335200.0|  794.02001953125|      GOOGLE|
|2017-01-06|807.9000244140625|792.2039794921875| 795.260009765625|806.1500244140625|1640200.0|806.1500244140625|      GOOGLE|
|2017-01-09|809.9660034179688|802.8300170898438|806.4000244140625|806.6500244140625|1274600.0|806.6500244140625|      

### Regression

In [26]:
trainDF, testDF = df.randomSplit([.8, .2], seed=42)
print(f"There are {trainDF.cache().count()} rows in the training set, and {testDF.cache().count()} in the test set")

There are 828 rows in the training set, and 159 in the test set


22/06/14 12:33:46 WARN CacheManager: Asked to cache already cached data.
22/06/14 12:33:46 WARN CacheManager: Asked to cache already cached data.


In [29]:
(trainRepartitionDF, testRepartitionDF) = (df
                                           .repartition(24)
                                           .randomSplit([.8, .2], seed=42))
print(f"There are {trainRepartitionDF.cache().count()} rows in the training set, and {testRepartitionDF.cache().count()} in the test set")

22/06/14 12:34:59 WARN CacheManager: Asked to cache already cached data.
22/06/14 12:34:59 WARN CacheManager: Asked to cache already cached data.


There are 784 rows in the training set, and 203 in the test set


In [34]:
(trainDF.select("Volume", "Close").summary()).show()

+-------+------------------+------------------+
|summary|            Volume|             Close|
+-------+------------------+------------------+
|  count|               828|               828|
|   mean|1629075.5084541063|1164.7888630300329|
| stddev| 706348.5354362321|215.30362136420584|
|    min|          285821.0| 786.1400146484375|
|    25%|         1193500.0|            1025.5|
|    50%|         1446500.0| 1132.800048828125|
|    75%|         1843400.0| 1263.699951171875|
|    max|         5167700.0|1807.1199951171875|
+-------+------------------+------------------+



In [59]:
from pyspark.ml.feature import VectorAssembler


# C'est faux ! Mais on fait ça pour avoir une idée de ce que l'on peut faire!
vecAssembler = VectorAssembler(inputCols=["Volume", "Open"], outputCol="features")

vecTrainDF = vecAssembler.transform(trainDF)

vecTrainDF.select("features", "Close").show(10)

+--------------------+-----------------+
|            features|            Close|
+--------------------+-----------------+
|[1657300.0,778.80...|786.1400146484375|
|[1073000.0,788.35...|786.9000244140625|
|[1640200.0,795.26...|806.1500244140625|
|[1274600.0,806.40...|806.6500244140625|
|[1176800.0,807.85...|804.7899780273438|
|[1353100.0,807.14...|806.3599853515625|
|[1362100.0,807.08...|804.6099853515625|
|[1294400.0,805.80...|806.0700073242188|
|[919300.0,805.119...|802.1749877929688|
|[1670000.0,806.90...|  805.02001953125|
+--------------------+-----------------+
only showing top 10 rows



In [60]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="features", labelCol="Close")
#lrModel = lr.fit(vecTrainDF)

In [62]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vecAssembler, lr])
pipelineModel = pipeline.fit(trainDF)

22/06/14 12:52:18 WARN Instrumentation: [65402247] regParam is zero, which might cause numerical instability and overfitting.


In [63]:
predDF = pipelineModel.transform(testDF)

predDF.select("Close", "prediction").show(10)

+-----------------+-----------------+
|            Close|       prediction|
+-----------------+-----------------+
|  794.02001953125| 788.046447971487|
|807.9099731445312|807.5158913894134|
|807.8800048828125|809.9208106445692|
|819.3099975585938|807.8378142630181|
|796.7899780273438|797.0407586626543|
|801.3400268554688|801.9694418669648|
|820.4500122070312|821.5192276807949|
|831.3300170898438|831.7263528499154|
|838.6799926757812|838.0507203871839|
|           843.25|844.3712353335196|
+-----------------+-----------------+
only showing top 10 rows



Series temporelles.

regression